In [1]:
import math,random,os
import numpy as np
from scipy.optimize import leastsq
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
from numba import jit

# Ising

In [2]:
@jit
def ising():
    Ts = [1,1.2,1.4,1.6,1.8,2.0,2.1,2.2,2.3,2.4,2.6,2.8,3,3.25,3.5,3.75,4,4.25,4.5,4.75,5]
    inpf = open('inp.txt','r')
    for line in inpf:
        L,eqsteps,nbins,nsamples = line.split()
        for T in Ts:
            LT_ising(int(L),float(T),int(eqsteps),int(nbins),int(nsamples))
    inpf.close()

In [3]:
@jit
def LT_ising(L,T,eqsteps,nbins,nsamples):
    
    N,beta,S,nbr = init(L,T)
    
    S = eq(N,beta,S,nbr,eqsteps)
    
    for ib in range(nbins):
        
        wirte_pro(L,T,ib)
        
        m_av,m2_av,E_av,E2_av = 0,0,0,0
        
        for isa in range(nsamples):
            S = heat_bath(N,beta,S,nbr)
            m_av,m2_av,E_av,E2_av = observable(N,S,nbr,m_av,m2_av,E_av,E2_av)
        
        writebin(m_av,m2_av,E_av,E2_av,nsamples,L,T)

In [4]:
def init(L,T):
    N = L * L
    beta = 1 / T
    
    conf = np.full(N,1)
    nbr = {i : ((i // L) * L + (i + 1) % L, (i + L) % N,
            (i // L) * L + (i - 1) % L, (i - L) % N) \
                                    for i in range(N)}
    
    return N,beta,conf,nbr

In [5]:
def metroplis(N,beta,conf,nbr):
    k = random.randint(0, N - 1)
    delta_E = 2 * conf[k] * np.sum(conf[nb] for nb in nbr[k])
    if random.uniform(0, 1) < math.exp(-beta * delta_E):
        conf[k] *= -1
    return conf

In [6]:
@jit
def heat_bath(N,beta,conf,nbr):
    k = random.randint(0, N - 1)
    Upsilon = random.uniform(0.0, 1.0)
    h = sum(S[nn] for nn in nbr[k])
    Sk_old = S[k]
    S[k] = -1
    if Upsilon < 1.0 / (1.0 + math.exp(-2.0 * beta * h)):
        S[k] = 1
    
    k = random.randint(0, N - 1)
    new_conf = conf[:]
    new_conf[k] *= -1
    E_old = energy(conf,N,nbr)
    E_new = energy(conf,N,nbr)
    if random.uniform(0,1) < E_new / (E_new + E_old):
        conf = new_conf
    return conf

In [7]:
@jit
def wollf(N,beta,conf,nbr):
    p  = 1.0 - math.exp(-2.0 * beta)
    k = random.randint(0, N - 1)
    
    Pocket, Cluster = [k], [k]
    while Pocket != []:
        j = random.choice(Pocket)
        for l in nbr[j]:
            if conf[l] == conf[j] and l not in Cluster \
                   and random.uniform(0,1) < p:
                Pocket.append(l)
                Cluster.append(l)
        Pocket.remove(j)

    for j in Cluster:
        conf[j] *= -1
    
    return conf

In [8]:
@jit
def eq(N,beta,conf,nbr,eqsteps):
    for eqstep in range(eqsteps):
        conf = heat_bath(N,beta,conf,nbr)
    return conf

In [9]:
def wirte_pro(L,T,ib):
    pf = open('process.txt','a')
    pf.write('running L={} T={} ib = {}'.format(L,T,ib) + '\n')
    pf.close()

In [10]:
@jit
def observable(N,conf,bnr,m_av,m2_av,E_av,E2_av):
    m_tot = np.sum(conf)
    rem = abs(m_tot) / N
    nbs = energy(conf,N,bnr)
    enr = nbs / N
    
    m_av +=rem
    m2_av += rem*rem
    E_av += enr
    E2_av += enr*enr
    
    return m_av,m2_av,E_av,E2_av

In [11]:
def energy(conf,N,nbr):
    E = 0
    for n in range(N):
         E -=  conf[n] * np.sum(conf[nb] for nb in nbr[n])
    return 0.5 * E

In [12]:
@jit
def writebin(m_av,m2_av,E_av,E2_av,nsamples,L,T):
    m_av = m_av / nsamples
    m2_av = m2_av / nsamples
    q = m2_av / (m_av**2)
    
    E_av = E_av / nsamples
    E2_av = E2_av / nsamples
    cv = (E2_av - E_av**2) * (L*L / T**2)
    
   
    bin = open('{}_bin.csv'.format(L),'a')
    strbin = str(L) + ',' + str(T) + ',' \
            + str(m_av) + ',' + str(E_av) + ','  \
            + str(cv) + ',' + str(q) 
    bin.write(strbin + '\n')
    bin.close()

# Result

In [13]:
@jit
def res():
    inte_dat()
    bin = pd.read_csv('bin.csv',index_col=False,dtype=float)
    Ls = [2,4,8,16,32,64]
    for L in Ls:
        result(L,bin)

In [14]:
def inte_dat():
    datf = open('bin.csv','a')
    Ls = [2,4,8,16,32,64]
    datf.write('1'+','+'2' + ',' + '3' + ',' + '4' + ',' + '5' + ',' + '6' + '\n')
    for L in Ls:
        dat_Lf = open('{}_bin.csv'.format(L),'r')
        dat_L = dat_Lf.read()
        datf.write(dat_L)
        dat_Lf.close()

In [15]:
@jit
def result(L,bin):
    wpr(L)
    nb = 10
    df = bin[bin['1']==L]
    Ts = [1,1.2,1.4,1.6,1.8,2.0,2.1,2.2,2.3,2.4,2.6,2.8,3,3.25,3.5,3.75,4,4.25,4.5,4.75,5]
    for T in Ts:
        df_t = df[df['2']==T]
        ds_t = df_t.describe()
        write_result(ds_t,T,L)

In [16]:
def wpr(L):
    pf = open('process.txt','a')
    pf.write('resulting L = {}'.format(L) + '\n')
    pf.close()

In [17]:
def write_result(ds_t,T,L):
    write_m(ds_t,T,L)
    write_e(ds_t,T,L)
    write_cv(ds_t,T,L)
    write_q(ds_t,T,L) 

In [18]:
def write_m(ds_t,T,L):
    f = open('m_{}.dat'.format(L),'a')
    f.write(str(T) + ',' + str(ds_t['3']['mean']) + ',' + str(ds_t['3']['std']) + '\n')    
    f.close()

In [19]:
def write_e(ds_t,T,L):
    f = open('E_{}.dat'.format(L),'a')
    f.write(str(T) + ',' + str(ds_t['4']['mean']) + ',' + str(ds_t['4']['std']) + '\n')    
    f.close()

In [20]:
def write_cv(ds_t,T,L):
    f = open('c_{}.dat'.format(L),'a')
    f.write(str(T) + ',' + str(ds_t['5']['mean']) + ',' + str(ds_t['5']['std']) + '\n')    
    f.close()

In [21]:
def write_q(ds_t,T,L):
    f = open('q_{}.dat'.format(L),'a')
    f.write(str(T) + ',' + str(ds_t['6']['mean']) + ',' + str(ds_t['6']['std']) + '\n')    
    f.close()

# Plot

In [22]:
@jit
def plot():
    superstition()
    
    a_s = ['m','E','c','q']
    for a in a_s:
        plot_a(a)
        
    finish()

In [23]:
def superstition():
    pf = open('process.txt','a')
    pf.write('FINALLY ploting' + '\n')
    pf.write('God Bless Me!!!' + '\n')
    pf.close()

In [24]:
def plot_a(a):
    plt.figure(dpi=1024)
    plt.xlabel('$T$',fontsize=14)
    plt.ylabel('${}$'.format(a),fontsize=14)
    plt.title('${}-T$'.format(a),fontsize=14)
    
    Ls = [2,4,8,16,32,64]
    for L in Ls:
        Ts,a_avs,a_sigmas = read_res(a,L)
        plt.errorbar(Ts,a_avs,yerr=a_sigmas,label='${}\\times{}$'.format(L,L))
        
    plt.legend(fontsize='large')
    plt.savefig('{}.png'.format(a),dpi=1024)

In [25]:
@jit
def read_res(a,L):
    Ts,a_avs,a_sigmas = [],[],[]
    f = open('{}_{}.dat'.format(a,str(L)))
    for line in f:
        T,a_av,a_sigma = line.split(',')
        Ts.append(float(T))
        a_avs.append(float(a_av))
        a_sigmas.append(float(a_sigma))
    f.close()
    return Ts,a_avs,a_sigmas

In [26]:
def finish():
    pf = open('process.txt','a')
    pf.write('FINISHED' + '\n')
    
    a = input("right or wrong?")
    pf.write(a)
    
    pf.close()

# fit

In [27]:
def fit():
    q,T,L = read_q()
    p0=[2.5,1,1,1,1,1,1]
    Para=leastsq(error,p0,args=(T,L,q,s))
    write_fit(Para)

In [28]:
@jit
def read_q():
    Ls = np.array([2,4,8,16,32,64])
    Ts = np.array([1.8,2.0,2.1,2.2,2.3,2.4,2.6,2.8])
    q  = np.empty((len(Ls),len(Ts)))
    for L in Ls:
        qlf = open('q_{}.dat'.format(L))
        for line in qlf:
            T,q_av,sigma_q = line.split()
            q[Ts.index(T),Ls.index(L)] = q_av
    return q,Ts,Ls

In [29]:
def func(p,T,L):
    Tc,qc,a0,v,a1,u1,w1 = p
    return qc + a0 * (T - Tc) * L**(1/v) +a1 * u1 * L**(-w1)

In [30]:
def fit_error(p,T,L,q,s='Test the number of iteration'):
    #print (s)
    return func(p,T,L)-q

In [31]:
def write_fit(Para):
    p_str = ['Tc','qc','a0','v','a1','u1','w1']
    fitf = open('fit.txt','w')
    for ip in p_str:
        fitf.write(ip + str(Para[p_str.index(p)]) + '\n')
    fitf.close()

# Run

In [32]:
def run():
    ising()
    res()
    plot()

In [33]:
run()

C:\Users\zhang\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in double_scalars
  


KeyboardInterrupt: 